In [1]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from criterion.cross_entropy import CrossEntropy
from numpy_modules.inception_v3 import InceptionV3
from datasets import cars
from utils import get_batches
from IPython import display
from ada_smooth import ada_smooth_optimizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy.typing as npt

In [2]:
x_train, y_train, x_test, y_test = cars.load_dataset(image_size=(75, 75))

100%|██████████| 8144/8144 [01:00<00:00, 135.71it/s]


In [3]:
epochs = 1
batch_size = 8
num_classes = 196

In [4]:
def one_hot(labels: npt.NDArray, num_classes: int = num_classes):
    result = np.zeros((labels.size, num_classes))
    result[np.arange(labels.size), labels] = 1
    return result


y_test_labels = y_test
y_train = one_hot(y_train)
y_test = one_hot(y_test)

In [5]:
model = InceptionV3(num_classes=num_classes)
model.train()

criterion = CrossEntropy()

optimizer_config = {
    "learning_rate": 1e-3,
    "epsilon": 1e-6,
    "fast_decay": 0.5,
    "slow_decay": 0.99,
}
optimizer_state = {}

loss_history = []
f1_micro = []

step = 1
for epoch in range(1, epochs + 1):
    for x_batch, y_batch in tqdm(
            get_batches((x_train, y_train), batch_size),
            total=len(x_train) // batch_size,
    ):
        model.zero_grad_parameters()

        predictions, _ = model.forward(x_batch)

        loss = criterion.forward(predictions, y_batch)

        dp = criterion.backward(predictions, y_batch)
        model.backward(x_batch, dp)

        parameters = model.get_parameters()[0]
        ada_smooth_optimizer(
            model.get_parameters(),
            model.get_grad_parameters(),
            optimizer_config,
            optimizer_state,
        )

        loss_history.append(loss)

        # display.clear_output(wait=True)
        #
        # f = plt.figure()
        # plt.plot(loss_history)
        # f.suptitle('training loss', fontsize=18)
        # plt.xlabel('iteration', fontsize=14)
        # plt.ylabel('loss', fontsize=14)
        # plt.show()
        # f.savefig('loss.jpg')

  0%|          | 0/918 [00:00<?, ?it/s]

[[[[ 0.3982993   0.15027298  0.36516873]
   [ 0.10826901 -0.47372202 -0.319649  ]
   [-0.43992797  0.47080209  0.03267034]]

  [[ 0.45653051 -0.53669106  0.26007368]
   [-0.50002691  0.39258785  0.38112313]
   [ 0.17756552 -0.06374705 -0.46679001]]

  [[ 0.27803684  0.25817946  0.31345368]
   [ 0.40614947 -0.49701873  0.15136589]
   [-0.09856108  0.34951553 -0.51648955]]]


 [[[ 0.08686108 -0.33744083 -0.08128126]
   [-0.21120322 -0.29441815  0.06161171]
   [-0.02695747 -0.35948534 -0.21340365]]

  [[-0.12004799  0.50069609 -0.34371057]
   [ 0.57477833 -0.01795753 -0.41132847]
   [-0.02809586  0.44552503  0.13020383]]

  [[-0.17337993  0.48058344 -0.45451145]
   [ 0.1887637  -0.17085006  0.12652605]
   [-0.19377292  0.1264158  -0.27897789]]]


 [[[ 0.2373146   0.46144062 -0.00102394]
   [-0.41281738  0.2911367   0.0429259 ]
   [-0.4658232   0.3943316   0.03949223]]

  [[-0.32565144  0.549095   -0.43226351]
   [ 0.14892539  0.44067664  0.46850924]
   [-0.3713719   0.16099483 -0.35630305

  0%|          | 1/918 [00:34<8:44:45, 34.34s/it]

[[[[ 0.7965986   0.30054595  0.73033746]
   [ 0.21653802 -0.94744404 -0.639298  ]
   [-0.87985594  0.94160418  0.06534067]]

  [[ 0.91306102 -1.07338212  0.52014736]
   [-1.00005383  0.7851757   0.76224626]
   [ 0.35513105 -0.12749409 -0.93358002]]

  [[ 0.55607369  0.51635891  0.62690735]
   [ 0.81229894 -0.99403745  0.30273179]
   [-0.19712215  0.69903107 -1.0329791 ]]]


 [[[ 0.17372217 -0.67488166 -0.16256253]
   [-0.42240644 -0.58883631  0.12322343]
   [-0.05391495 -0.71897067 -0.42680731]]

  [[-0.24009597  1.00139217 -0.68742113]
   [ 1.14955667 -0.03591507 -0.82265694]
   [-0.05619171  0.89105006  0.26040766]]

  [[-0.34675986  0.96116687 -0.90902291]
   [ 0.37752741 -0.34170012  0.25305211]
   [-0.38754585  0.2528316  -0.55795579]]]


 [[[ 0.47462919  0.92288124 -0.00204788]
   [-0.82563475  0.58227339  0.0858518 ]
   [-0.9316464   0.78866321  0.07898445]]

  [[-0.65130287  1.09818999 -0.86452703]
   [ 0.29785077  0.88135328  0.93701847]
   [-0.7427438   0.32198967 -0.71260611

  0%|          | 2/918 [01:08<8:46:06, 34.46s/it]

...

[[[[ 4.77959159  1.80327572  4.38202477]
   [ 1.29922811 -5.68466426 -3.83578801]
   [-5.27913562  5.64962505  0.39204405]]

  [[ 5.4783661  -6.44029275  3.12088415]
   [-6.00032296  4.7110542   4.57347753]
   [ 2.1307863  -0.76496456 -5.60148014]]

  [[ 3.33644212  3.09815347  3.76144413]
   [ 4.87379365 -5.96422471  1.81639071]
   [-1.18273292  4.19418639 -6.19787459]]]


 [[[ 1.04233301 -4.04928997 -0.97537516]
   [-2.53443861 -3.53301785  0.73934056]
   [-0.3234897  -4.31382405 -2.56084383]]

  [[-1.44057585  6.00835304 -4.12452681]
   [ 6.89734    -0.21549041 -4.93594166]
   [-0.33715029  5.34630038  1.56244598]]

  [[-2.08055914  5.76700124 -5.45413744]
   [ 2.26516445 -2.05020071  1.51831265]
   [-2.32527507  1.51698959 -3.34773474]]]


 [[[ 2.84777514  5.53728743 -0.01228725]
   [-4.95380852  3.49364037  0.51511082]
   [-5.5898784   4.73197926  0.47390672]]

  [[-3.90781723  6.58913995 -5.18716216]
   [ 1.78710464  5.2881197   5.62211084]
   [-4.45646281  1.93193801 -4.27563664

  1%|▏         | 12/918 [06:54<8:20:00, 33.11s/it]

[[[[ 5.17789088  1.95354869  4.7471935 ]
   [ 1.40749712 -6.15838629 -4.15543701]
   [-5.71906359  6.12042714  0.42471438]]

  [[ 5.9348966  -6.97698381  3.38095783]
   [-6.50034988  5.10364205  4.95460066]
   [ 2.30835182 -0.8287116  -6.06827015]]

  [[ 3.61447896  3.35633293  4.0748978 ]
   [ 5.27994312 -6.46124344  1.9677566 ]
   [-1.28129399  4.54370193 -6.71436414]]]


 [[[ 1.12919409 -4.3867308  -1.05665643]
   [-2.74564183 -3.82743601  0.80095228]
   [-0.35044717 -4.67330939 -2.77424748]]

  [[-1.56062383  6.50904913 -4.46823737]
   [ 7.47211833 -0.23344794 -5.34727013]
   [-0.36524614  5.79182541  1.69264981]]

  [[-2.25393907  6.24758468 -5.90864889]
   [ 2.45392815 -2.22105077  1.6448387 ]
   [-2.519048    1.64340539 -3.62671263]]]


 [[[ 3.08508974  5.99872805 -0.01331119]
   [-5.3666259   3.78477706  0.55803672]
   [-6.0557016   5.12631086  0.51339895]]

  [[-4.23346866  7.13823495 -5.61942568]
   [ 1.93603002  5.72879634  6.09062008]
   [-4.82783471  2.09293285 -4.6319397 

  1%|▏         | 13/918 [07:47<9:02:34, 35.97s/it]


KeyboardInterrupt: 

In [ ]:
model.evaluate()
predictions = model.forward(x_test)
accuracy_score = accuracy_score(y_test_labels, predictions.argmax(axis=1))
precision_score = precision_score(y_test_labels, predictions.argmax(axis=1),average="micro")
recall_score = recall_score(y_test_labels, predictions.argmax(axis=1),average="micro")
f1_score = f1_score(y_test_labels, predictions.argmax(axis=1),average="micro")

print(f"accuracy: {accuracy_score}\nprecision: {precision_score}\nrecall: {recall_score}\nf1: {f1_score}")